In [ ]:
import sys
sys.path.insert(1, '/home/mi/thempel/mystorage/software/pyemma/')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import mdshare
import pyemma

In [ ]:
pdb = mdshare.fetch('pentapeptide-impl-solv.pdb', working_directory='data')
files = mdshare.fetch('pentapeptide-*-500ns-impl-solv.dcd', working_directory='data')

In [ ]:
lag = 10
dim = 10

torsions_feat = pyemma.coordinates.featurizer(pdb)
torsions_feat.add_backbone_torsions(cossin=True)
torsions_data = pyemma.coordinates.load(files, features=torsions_feat)
torsions_vamp = pyemma.coordinates.vamp(torsions_data, lag=lag, dim=dim)
scores = [torsions_vamp.score()]
labels = ['backbone\ntorsions']

positions_feat = pyemma.coordinates.featurizer(pdb)
positions_feat.add_selection(positions_feat.select_Backbone())
positions_data = pyemma.coordinates.load(files, features=positions_feat)
positions_vamp = pyemma.coordinates.vamp(positions_data, lag=lag, dim=dim)
scores += [positions_vamp.score()]
labels += ['backbone atom\npositions']

distances_feat = pyemma.coordinates.featurizer(pdb)
distances_feat.add_distances(
    distances_feat.pairs(distances_feat.select_Backbone(), excluded_neighbors=2))
distances_data = pyemma.coordinates.load(files, features=distances_feat)
distances_vamp = pyemma.coordinates.vamp(distances_data, lag=lag, dim=dim)
scores += [distances_vamp.score()]
labels += ['backbone atom\ndistances']

plt.bar(labels, scores)
plt.ylabel('VAMP2 score')
vamp_bars_plot = dict(labels=labels, scores=scores, dim=dim, lag=lag) # save for later

In [ ]:
lags = [1, 2, 5, 10, 20, 50]
dims = [i + 1 for i in range(10)]

#vamp_conv_plot = dict() # save for later
for lag in lags:
    scores = [pyemma.coordinates.vamp(torsions_data, lag=lag, dim=dim).score()
              for dim in dims]
    plt.plot(dims, scores, label='lag={:.1f}ns'.format(lag * 0.1))
    #vamp_conv_plot.update()
plt.legend()

In [ ]:
tica = pyemma.coordinates.tica(torsions_data, lag=5)
tica_all = np.concatenate(tica.get_output())
tica_output = tica.get_output()
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(tica_all, ax=axes[0])
pyemma.plots.plot_density(*tica_all.T[:2], ax=axes[1], logscale=True)
fig.tight_layout()

Let’s have a look how one of the trajectories looks like in the space of the first three TICA components. We can see that the TICA components nicely resolve the slow transitions as discrete jumps.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(8,5))
x = 0.1 * np.arange(tica_output[0].shape[0])
axes[0].plot(x, tica_output[0][:,0])
axes[1].plot(x, tica_output[0][:,1])
axes[2].plot(x, tica_output[0][:,2])
axes[2].set_xlabel('time / ns')
for n, ax in enumerate(axes):
    ax.set_ylabel('IC {}'.format(n+1))
fig.tight_layout()

The TICA coordinates are now clustered into a number of discrete states using the $k$-means algorithm. The $k$-means algorithm requires as input the number of clusters. The trajectories are automatically assigned to the cluster centers by calling `cluster.dtrajs`. We can check how this works in the first two TICA dimensions with

In [ ]:
cluster = pyemma.coordinates.cluster_kmeans(tica, k=200, max_iter=50, stride=10)

fig, ax = plt.subplots()
pyemma.plots.plot_density(*tica_all.T[:2], ax=ax, cbar=False, alpha=0.1)
ax.scatter(*cluster.clustercenters.T[:2], s=15, c='C1')

The states are well distributed in phase space.


Now, we calculate the implied timescales with `pyemma.msm.its()` up to a lagtime of $50$ steps (as defined by `lags=50`). Instead of a single number, an array can be passed to compute the ITS at defined lagtimes. We directly estimate the error bars with Bayesian MSMs. To speed up the computation, this can be disabled.

In [ ]:
its = pyemma.msm.its(cluster.dtrajs, lags=50, nits=10, errors='bayes')
pyemma.plots.plot_implied_timescales(its, units='ns', dt=.1);

It can be seen that the timescales are constant within the error. Below we will select a lag time of $5$ steps ($0.5$ ns) to build a Markov model.The lagtime to estimate the Markov model is specified as `lag` here. As a quick check we print the fraction of states and counts that are in the active set.

In [ ]:
msm = pyemma.msm.bayesian_markov_model(cluster.dtrajs, lag=5, dt_traj='0.1 ns')
print('fraction of states used = {:f}'.format(msm.active_state_fraction))
print('fraction of counts used = {:f}'.format(msm.active_count_fraction))

The model is validated with a Chapman-Kolmogorov test. The number of metastable sets to test for is estimated from the implied timescales plot. We find three processes that can be resolved for up to $5$ ns lag times that display a relatively large gap towards the faster processes. To capture those, we need to define $4$ metastable states. This assumption is validated with the Chapman-Kolmogorov test.

In [ ]:
pyemma.plots.plot_cktest(msm.cktest(4));

The assumption of $4$ metastable states seems to hold.

From the MSM which is now stored in the object we called `msm`, various properties can be obtained. We start by analyzing the stationary distribution and the free energy computed over the first two TICA coordinates. The stationary distribution $\pi$ is stored in `msm.pi` or `msm.stationary_distribution`. We further compute the free energy landscape by re-weighting the trajectory frames with stationary probabilities from the MSM.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
pyemma.plots.plot_contour(*tica_all.T[:2], msm.pi[np.concatenate(cluster.dtrajs)], mask=True, ax=axes[0], cbar_label='stationary distribution')
pyemma.plots.plot_free_energy(*tica_all.T[:2], legacy=False, weights=np.concatenate(msm.trajectory_weights()), ax=axes[1])

Now we analyze the slowest processes by looking at the distribution of states along the first 3 eigenvectors.

In [ ]:
eigvec = msm.eigenvectors_right()
print('first eigenvector is one: {} (min={}, max={})'.format(
    np.allclose(eigvec[:, 0], 1, atol=1e-15), eigvec[:, 0].min(), eigvec[:, 0].max()))

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
for i, ax in enumerate(axes.flat):
    pyemma.plots.plot_contour(
        *tica_all.T[:2], eigvec[np.concatenate(cluster.dtrajs), i + 1], ax=ax, cmap='PiYG',
        cbar_label='{}. right eigenvector'.format(i + 2), mask=True)
    ax.set_xlabel('IC 1')
axes[0].set_ylabel('IC 2')
fig.tight_layout()

From the sign change of the eigenvectors we extract between which states the corresponding process happens. Since the eigenvectors were internally sorted according to their eigenvalue, they correspond to the slowest processes of the implied timescale plot.

Next, we do a coarse graining into a user defined number of macrostates. As already discussed, $4$ is a good choice for this example.

In [ ]:
msm.pcca(4)

We have now determined the probability for each microstate to belong to a given macrostate. These probabilities are called memberships to a given macrostate.

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 3))
for i, ax in enumerate(axes.flat):
    pyemma.plots.plot_contour(
        *tica_all.T[:2], msm.metastable_distributions[i][np.concatenate(cluster.dtrajs)], ax=ax, cmap='afmhot_r', 
        mask=True, method='nearest', cbar_label='metastable distribution {}'.format(i))
    ax.set_xlabel('IC 1')
axes[0].set_ylabel('IC 2')
fig.tight_layout()

For each macrostate we can generate a number of representative sample structures and store them into a trajectory file.

In [ ]:
pcca_samples = msm.sample_by_distributions(msm.metastable_distributions, 10)
torsions_source = pyemma.coordinates.source(files, features=torsions_feat)
pyemma.coordinates.save_trajs(torsions_source, pcca_samples,  
                              outfiles=['./data/pcca1_10samples.xtc','./data/pcca2_10samples.xtc',
                                        './data/pcca3_10samples.xtc','./data/pcca4_10samples.xtc'])

In [ ]:
#TODO: visualize metastable structures with nglview

In [ ]:
metastable_traj = msm.metastable_assignments[np.concatenate(cluster.dtrajs)]
highest_membership = msm.metastable_distributions.argmax(1)
coarse_state_centers = cluster.clustercenters[msm.active_set[highest_membership]]

In [ ]:
nstates = 4
mfpt = np.zeros((nstates, nstates))
for i in range(nstates):
    for j in range(nstates):
        mfpt[i, j] = msm.mfpt(
            msm.metastable_sets[i],
            msm.metastable_sets[j])

rate = np.zeros_like(mfpt)
nz = mfpt.nonzero()
rate[nz] = 1.0 / mfpt[nz]

In [ ]:
cmap = mpl.cm.get_cmap('viridis', nstates)

fig, ax = plt.subplots(figsize=(8, 6))
pyemma.plots.plot_state_map(*tica_all.T[:2], metastable_traj, ax=ax, zorder=-1)

pyemma.plots.plot_network(
    rate,
    pos=coarse_state_centers,
    figpadding=0,
    arrow_label_format='%.1f ps',
    arrow_labels=mfpt,
    size=20,
    state_scale=0,
    show_frame=True,
    ax=ax)

#ax.set_xlabel('$\Phi$')
#ax.set_ylabel('$\Psi$')
ax.set_xlim(-2, 8)
ax.set_ylim(-2, 8)
fig.tight_layout()

In [ ]:
A = msm.metastable_sets[2]
B = msm.metastable_sets[1]
flux = pyemma.msm.tpt(msm, A, B)
A, B

In [ ]:
flux.A

In [ ]:
cg, cgflux = flux.coarse_grain(msm.metastable_sets)
cgflux.I, cgflux.A, cgflux.B, cg

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

pyemma.plots.plot_contour(
    *tica_all.T[:2], flux.committor[np.concatenate(cluster.dtrajs)], cmap='brg', ax=ax,
    mask=True, method='nearest', cbar_label='committor 0 -> 3', alpha=0.8, zorder=-1);

pyemma.plots.plot_flux(cgflux, coarse_state_centers[[1, 0, 3, 2]], cgflux.stationary_distribution, ax=ax, 
                       show_committor=False, figpadding=0, show_frame=True, arrow_label_format='%2.e / ps')
ax.set_xlim(-2, 8)
ax.set_ylim(-2, 8)
fig.tight_layout()

In [ ]:
hmm = pyemma.msm.bayesian_hidden_markov_model(cluster.dtrajs, 4)